In [5]:
import pandas as pd

# CSV-Dateien laden (Pfad ggf. anpassen)
beiträge_df = pd.read_csv("../Data/data_v3/Beitraege.csv")
likes_df = pd.read_csv("../Data/data_v3/Likes.csv")
nutzer_realistisch_df = pd.read_csv("../Data/data_v3/Nutzer.csv")


In [6]:
# Likes mit Beiträgen verknüpfen
beitraege_mit_likes = likes_df.merge(beiträge_df, on="beitrag_id", how="left")

# Likes mit realistischen Nutzerdaten verknüpfen
likes_mit_nutzern = beitraege_mit_likes.merge(nutzer_realistisch_df, on="nutzer_id", how="left")

# Hashtags vorbereiten
likes_mit_nutzern["hashtags"] = likes_mit_nutzern["hashtags"].str.lower().str.replace("#", "", regex=False)
likes_mit_nutzern = likes_mit_nutzern.dropna(subset=["hashtags"])
likes_mit_nutzern["hashtag_einzeln"] = likes_mit_nutzern["hashtags"].str.split(" ")

# Einzelne Hashtags explodieren
likes_exploded = likes_mit_nutzern.explode("hashtag_einzeln")

# Analyse pro Hashtag
hashtag_auswertung_realistisch = likes_exploded.groupby("hashtag_einzeln").apply(
    lambda df: pd.Series({
        "gesamtanzahl": len(df),
        "nichtwaehler_anzahl": (df["wahlfrequenz"].str.lower() == "nie").sum(),
        "dominante_partei": df[df["wahlfrequenz"].str.lower() != "nie"]["gewaehlte_partei"].mode().iloc[0]
            if not df[df["wahlfrequenz"].str.lower() != "nie"]["gewaehlte_partei"].mode().empty else None
    })
).reset_index()

# Prozentualer Anteil der Nichtwähler berechnen
hashtag_auswertung_realistisch["anteil_nichtwaehler"] = (
    hashtag_auswertung_realistisch["nichtwaehler_anzahl"] /
    hashtag_auswertung_realistisch["gesamtanzahl"] * 100
).round(2)

# Optional: Nur Hashtags mit mindestens 10 Einträgen anzeigen
hashtag_auswertung_realistisch = hashtag_auswertung_realistisch[
    hashtag_auswertung_realistisch["gesamtanzahl"] >= 10
].sort_values(by="anteil_nichtwaehler", ascending=False)

# Ergebnisse anzeigen
print(hashtag_auswertung_realistisch.head(10))


   hashtag_einzeln  gesamtanzahl  nichtwaehler_anzahl dominante_partei  \
29          schule          1903                  658              SPD   
35          wahlen          1699                  587              SPD   
26       radfahren          1387                  479              SPD   
5   bildungfüralle          2038                  698              AfD   
7             chor          1576                  539              SPD   
14   gerechtigkeit          1775                  607              SPD   
6       blockchain          1880                  639              SPD   
32             uni          1859                  629              SPD   
21          lernen          1835                  617              AfD   
1         bandlife          1792                  601              SPD   

    anteil_nichtwaehler  
29                34.58  
35                34.55  
26                34.53  
5                 34.25  
7                 34.20  
14                34.20  
6  

/var/folders/7b/mkfxb1j17sd1h4hx_hq780c40000gn/T/ipykernel_20817/1566994169.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  hashtag_auswertung_realistisch = likes_exploded.groupby("hashtag_einzeln").apply(
